# Show top/worst n% of the sentences filtered by freq of word counts (in HT minus in MT) from _analysis table_

In [1]:
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager
from pprint import pprint

from jieba_postag_converter import get_postag_dict

def get_data_as_list(fpath) -> list:
    data = []
    with open(fpath, "r", encoding="utf-8-sig") as f:
        for line in f:
            line = line.replace("\n","").split(" ")
            data.append(line)
    return data

def get_diff_sentences(df,corpus="mpdd", situation='apology', sen_type='query', diff_type='del', percentile=75, top=True, thirdcol_obj='primitive form'):
    def count(x):
        return x.count() 
    df = df[df['corpus'].isin([corpus])&df['situation'].isin([situation])&df['sentence type'].isin([sen_type])&df['difference type'].isin([diff_type])]
    unaligned_count = df.groupby(['corpus', 'situation', 'method', 'sentence type', 'difference type','line'])['index'].apply(count).reset_index()
    unaligned_count = unaligned_count.rename(columns={'index': 'count'})
    
#     unaligned_count['quantile'] = unaligned_count.groupby(['corpus', 'situation', 'method', 'sentence type', 'difference type',])['count'].transform(lambda group: np.quantile(group,quantile))
#     print(unaligned_count)
#     if top==True:
#         diff_df = unaligned_count.groupby(['corpus', 'situation', 'method', 'sentence type', 'difference type','line']).filter(lambda group: group['count']>=group['quantile']).reset_index()
#     else:
#         diff_df = unaligned_count.groupby(['corpus', 'situation', 'method', 'sentence type', 'difference type','line']).filter(lambda group: group['count']<=group['quantile']).reset_index()

    unaligned_count.set_index('line',inplace=True)
    translated_dict = unaligned_count[unaligned_count['method']=='translated']['count'].to_dict()
    rewrited_dict = unaligned_count[unaligned_count['method']=='rewrited']['count'].to_dict()
    calced_dict={}
    for i in range(max(max(translated_dict),max(rewrited_dict))+1):
        if (i in translated_dict) and (i in rewrited_dict):
            calced_dict[i] = rewrited_dict[i] - translated_dict[i]
        elif (i not in translated_dict) and (i in rewrited_dict):
            calced_dict[i] = rewrited_dict[i]
        elif (i  in translated_dict) and (i not in rewrited_dict):
            calced_dict[i] = - translated_dict[i]
        else:
            pass
    values = list(calced_dict.values())
    threshold = np.percentile(np.array(values),percentile)
    calced_dict = dict(sorted(calced_dict.items(), key=lambda i: i[1], reverse=True))
#     print(calced_dict)
    method_type = ['original','translated','rewrited']
#     sentence_type = ['query', 'res']
    sentence_type = [sen_type]
    dirs = ['../mrphdata/']
    print(f'{corpus} {situation} {diff_type}')
    for line, val in calced_dict.items():
        if top==True and val < threshold:
            continue
        if top==False and val > threshold:
            continue
        for t in sentence_type:
            for m in method_type:
                for d in dirs:
                    path =f'{d}/{corpus}/{situation}/{m}_{t}'
                    with open(path, 'r', encoding='utf_8_sig')as f:
                        for i,sen in enumerate(f):
                            if i == line:
                                print('[{0:^5}] {1:<11} {2:<6}: {3}'.format(i,m,t,sen[:-1]))
        unaligned_tr = df[(df['line'].isin([line]))&(df['method'].isin(['translated']))]['word'].to_list()
        unaligned_re = df[(df['line'].isin([line]))&(df['method'].isin(['rewrited']))]['word'].to_list()
#             print(type(df))
        print('{0:<11}: {1}'.format('translated',' '.join(unaligned_tr)))
        print('{0:<11}: {1}'.format('rewrited',' '.join(unaligned_re)))
        print()
#     [diff_df['method']=='translated']['count'].to_dict()
#     print(translated_count)
#     for  g, item in diff_df:
#         print(g)
#         print(item)
#     return 0

In [3]:
t_path = "analysis_table.csv"
# t_path = "analysis_table_upper.csv"

df = pd.read_csv(t_path)

get_diff_sentences(df,'mpdd','apology','query','add',70,top=True)

mpdd apology add
[  2  ] original    query : 喂 ， 孙 校长 ， 真对不起 ， 我 先 回家 了 ， 没有 当 面向 您 请假 ， 请原谅 ， 你们 离开 芷江 了 没有 ？
[  2  ] translated  query : 孫 校長 さん 、 こんにちは 、 本当に 申し訳 あり ませ ん 、 先 に 帰り ます 、 私 が 直接 お 願い した わけで は あり ませ ん 、 許して ください 、 もう 紫 江 を 出 ました か ？
[  2  ] rewrited    query : もしもし 、 校長 先生 です か ？ 　 本当に すみません 。 お 先 に 失礼 いたし ました 。 事前 に ちゃんと 申請 せ ず に 休んで しまって 本当に 申し訳 ございませ ん 。 もう 芷 江 を お 出 に なり ました か ？
translated : 本当に 申し訳 ませ ん あり ませ ん
rewrited   : 校長 本当に お ちゃんと 申請 せ ず 休んで しまって 本当に 申し訳 ん もう 江 お なり

[ 98  ] original    query : 是 我 说 对不起 才 对 。 我 知道 你 对 我 的 感情 ， 但是 … … 感情 是 两个 人事 ， 你 明白 吗 ？
[ 98  ] translated  query : 謝る の は 当然の こと です 。 気持ち は わかる けど 、 ...... 気持ち は 2 人 の 人事 です 、 わかり ます か ？
[ 98  ] rewrited    query : 謝る の は 私 の 方 だ よ 。 そういう 風 に 思 わ てる って 知って た んだ から 。 でも 、 そういう の は 二 人 の こと だ から 、 ね ？
translated : の です
rewrited   : の 私 方 風 た でも そういう の 人 こと だ

[ 67  ] original    query : 对不起 ！ 我 又 让 你 难过 了 ！ 我 以后 不 逼 你 了 。 就 、 就让 一切 顺其自然 好 吗 ？
[ 67  ] translated  query : ごめん ね ！ また 悲しま せて しまった ！